# Datenanalyse der Brettspielwelt-DB

## Datenbestand

### Datenbeschaffung

Es stehen ca. 7 GB Logdateien vom Spiele-Portal "Brettspielwelt" zur Verfügung (seit 2007-01, alle Logdateien bis einschließlich 2025-07).
Jede Logdatei hat die Daten einer Partie.

Nach dem Download wurde jede Logdatei geparst (in ein Objekt eingelesen), validiert und in eine SQLite-DB gepumpt.
Dieser Vorgang dauerte 16 Stunden und 37 Minuten (ca. 25ms / Partie).

Dabei wurde gezählt:
*   2.411.514 Logdateien insgesamt
*   13.964 leere Logdateien (ohne verwertbare Daten)

Die SQLite-Datenbankdatei ist 14.842.700 KB = 14,155 GB groß.

### Anzahl Partien

*   2.397.550 Partien insgesamt
*   2.141.248 Partien fehlerfrei

In [21]:
%%sql
select count(*) as Partien from games;

,Partien
0,2397550


In [1]:
%%sql
select g.error_code as Code, e.name as Fehler, count(*) as Anzahl, e.description as Beschreibung
from games as g
left join errors as e on g.error_code = e.code
group by g.error_code
order by g.error_code;

,Code,Fehler,Anzahl,Beschreibung
0,0,NO_ERROR,2141248,Kein Fehler
1,70,GAME_NOT_FINISHED,76658,Partie nicht zu Ende gespielt
2,71,GAME_OVERPLAYED,1926,"Ein oder mehrere Runden gespielt, obwohl die P..."
3,80,ROUND_FAILED,21760,Mindestens eine Runde ist fehlerhaft
4,90,PLAYER_CHANGED,155958,Mindestens ein Spieler hat während der Partie ...


## Runden

*   22.458.253 Runden insgesamt.
*   22.416.517 Runden fehlerfrei
*   20.186.515 Runden unter den fehlerfreien Partien (durchschnittlich 9,4 Runden / Partie).

In [48]:
%%sql
select count(*) as Runden from rounds;

,Runden
0,22458253


In [6]:
%%sql
select r.error_code as Code, e.name as Fehler, count(*) as Anzahl, e.description as Beschreibung
from rounds as r
left join errors as e on r.error_code = e.code
group by r.error_code
order by r.error_code;

,Code,Fehler,Anzahl,Beschreibung
0,0,NO_ERROR,22416517,Kein Fehler
1,12,DUPLICATE_CARD,10,Karten mehrmals vorhanden
2,21,WISH_NOT_FOLLOWED,209,Wunsch nicht beachtet
3,23,SMALLER_OF_AMBIGUOUS_RANK,21267,Es wurde der kleinere Rang bei einem mehrdeuti...
4,24,PLAYER_NOT_ON_TURN,2,Der Spieler ist nicht am Zug
5,26,HISTORY_TOO_LONG,60,"Karten ausgespielt, obwohl die Runde vorbei is..."
6,31,DRAGON_GIVEN_TO_OWN_TEAM,306,Drache an eigenes Team verschenkt
7,50,ANNOUNCEMENT_NOT_POSSIBLE,286,Tichu-Ansage an der geloggten Position nicht m...
8,60,SCORE_NOT_POSSIBLE,17234,Rechenfehler! Geloggtes Rundenergebnis ist nic...
9,61,SCORE_MISMATCH,2362,Geloggtes Rundenergebnis stimmt nicht mit dem ...


In [4]:
%%sql
select count(*) as Runden
from rounds as r
inner join games as g on g.id = r.game_id
where g.error_code = 0;

,Runden
0,20186515


In [3]:
20186515 / 2141248

9.427453055414412

### Spieler

*   128.462 Spieler insgesamt
*   110.932 Spieler, die eine fehlerfreie Partie durchgespielt haben

In [9]:
%%sql
select count(*) as Spieler from players;

,Spieler
0,128462


In [5]:
%%sql
select count(*)
from (
select player_id_0 from games where error_code = 0
union
select player_id_1 from games where error_code = 0
union
select player_id_2 from games where error_code = 0
union
select player_id_3 from games where error_code = 0
) as sub;


,count(*)
0,110932
